In [23]:
!pip install pyvirtualdisplay Pillow


In [ ]:
import os

SUMO_HOME = 'C:\\Program Files (x86)\\Eclipse\\Sumo'

os.environ['SUMO_HOME'] = SUMO_HOME

# Print the SUMO_HOME environment variable to verify
print("SUMO_HOME is set to:", os.environ.get('SUMO_HOME'))

# Check if the SUMO_HOME path exists
print("Does SUMO_HOME exist?", os.path.exists(os.environ.get('SUMO_HOME')))


SUMO_HOME is set to: C:\Program Files (x86)\Eclipse\Sumo
Does SUMO_HOME exist? True


In [28]:
import gymnasium as gym
import sumo_rl
import os

# Print SUMO_HOME for confirmation
print("SUMO_HOME is set to:", os.environ['SUMO_HOME'])

# Define file paths
net_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.net.xml/osm.net.xml'
route_file = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.rou.xml'
out_csv_name = 'C:/Users/ASUS/Sumo/2024-11-03-18-23-47/osm.passenger.csv'

# Initialize SUMO Gym environment
env = gym.make(
    'sumo-rl-v0',
    net_file=net_file,
    route_file=route_file,
    out_csv_name=out_csv_name,
    use_gui=True,
    num_seconds=100000
)

# Reset the environment and start simulation
obs, info = env.reset()
done = False

while not done:
    next_obs, reward, terminated, truncated, info = env.step(env.action_space.sample())
    print("Reward:", reward)
    print("Observation:", next_obs)
    done = terminated or truncated


SUMO_HOME is set to: C:\Program Files (x86)\Eclipse\Sumo
 Retrying in 1 seconds
Reward: 0.0
Observation: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observation: [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Reward: 0.0
Observat

KeyboardInterrupt: 

In [29]:
import gymnasium as gym
import traci
from gymnasium import spaces
import numpy as np

class CustomSumoEnv(gym.Env):
    def __init__(self, net_file, route_file, use_gui=False):
        super(CustomSumoEnv, self).__init__()
        
        # Initialize SUMO parameters
        self.net_file = net_file
        self.route_file = route_file
        self.use_gui = use_gui
        
        # Define observation and action spaces for each agent (multi-agent setup)
        self.observation_space = spaces.Box(low=0, high=1, shape=(4,), dtype=np.float32)  # Example observation space
        self.action_space = spaces.Discrete(3)  # Example action space (3 discrete actions)

        # Initialize SUMO simulation
        self._init_sumo()

    def _init_sumo(self):
        """Initialize the SUMO simulation."""
        sumo_binary = "sumo-gui" if self.use_gui else "sumo"
        sumo_cmd = [sumo_binary, "-n", self.net_file, "-r", self.route_file]
        
        traci.start(sumo_cmd)

    def reset(self):
        """Reset the environment."""
        traci.load([self.net_file, self.route_file])
        
        # Reset observations for each agent
        obs = self._get_observations()
        
        return obs

    def _get_observations(self):
        """Get observations for each agent."""
        # Example: Get vehicle speeds or positions from SUMO via TraCI
        vehicle_ids = traci.vehicle.getIDList()
        observations = {}
        
        for vehicle_id in vehicle_ids:
            speed = traci.vehicle.getSpeed(vehicle_id)
            position = traci.vehicle.getPosition(vehicle_id)
            observations[vehicle_id] = np.array([speed, position[0], position[1], 1.0])  # Example observation
        
        return observations

    def step(self, actions):
        """Take a step in the environment."""
        
        # Execute actions for each agent (e.g., change speed or route)
        for vehicle_id, action in actions.items():
            if action == 0:
                traci.vehicle.setSpeed(vehicle_id, 10)  # Example action: set speed to 10 m/s
            elif action == 1:
                traci.vehicle.setSpeed(vehicle_id, 20)  # Example action: set speed to 20 m/s
            elif action == 2:
                traci.vehicle.setSpeed(vehicle_id, 30)  # Example action: set speed to 30 m/s
        
        traci.simulationStep()  # Advance the simulation by one step
        
        # Get new observations and rewards after taking actions
        obs = self._get_observations()
        rewards = self._compute_rewards()
        
        done = False  # Define your termination condition here

        return obs, rewards, done, {}

    def _compute_rewards(self):
        """Compute rewards for each agent."""
        rewards = {}
        
        vehicle_ids = traci.vehicle.getIDList()
        
        for vehicle_id in vehicle_ids:
            speed = traci.vehicle.getSpeed(vehicle_id)
            rewards[vehicle_id] = speed / 30.0  # Example reward: normalize speed
        
        return rewards

    def close(self):
        """Close the SUMO simulation."""
        traci.close()


In [ ]:
env = CustomSumoEnv(net_file, route_file, use_gui=True)


FileNotFoundError: [WinError 2] The system cannot find the file specified